# Report Analytics

## Transformacao e enriquecimento da base

######  1 - Importando os dados do CSV

In [1]:
import sqlite3
import pandas as pd
from pandasql import sqldf
import pandasql as ps
import random

In [2]:
nome = pd.read_csv('Data/Nome.csv', sep=';') 
estado = pd.read_csv('Data/ESTADO.csv', sep=';')

######  2 - Duplicando os dados

In [3]:
#duplicando os dados aleatoriamente
novo_nome = nome.sample(n=500,replace=True)
novo_nome = novo_nome.reset_index(drop=True)
novo_estado = estado.sample(n=500,replace=True)
novo_estado = novo_estado.reset_index(drop=True)

In [4]:
#vericando a quantidade
novo_nome.tail()

,NOME,SOBRENOME,IDADE
495,ERICK,CARVALHO,44
496,CARLOS,GARCIA,36
497,FRANCISCO,FREITAS,18
498,RAQUEL,GOMES,30
499,JOAO,FERREIRA,26


In [5]:
#vericando a quantidade
novo_estado.tail()

,CIDADE,ESTADO,PAIS,NOME,SOBRENOME
495,Batalha,AL,BRASIL,JESSICA,ALVES
496,Anahy,PR,BRASIL,RAQUEL,GOMES
497,Bom Jardim,RJ,BRASIL,RAFAEL,LIMA
498,Aloândia,GO,BRASIL,ERICK,CARVALHO
499,Brasília,DF,BRASIL,JULIANA,CAMPOS


######  3 - Criando um banco de dados e transformando os arquivos em CSV para tabelas SQL

In [6]:
#conectando ao banco de dados e transformando os arquivos CSV em tabelas SQL
conn = sqlite3.connect('desafafio2.db')
novo_nome.to_sql('novo_nomeSQL', conn, if_exists='replace')
novo_estado.to_sql('novo_estadoSQL', conn, if_exists='replace')

500

######  4 - Integração das duas tabelas

In [7]:
#juntando os dados das duas tabelas
resultado1 = pd.read_sql_query(
    '''SELECT n.nome, n.sobrenome, n.idade, e.cidade, e.estado, e.pais
       FROM novo_nomeSQL as n
       LEFT JOIN novo_estadoSQL as e 
         on n.NOME = e.NOME
         and n.Sobrenome = e.Sobrenome
       ORDER BY n.nome''', conn)
conn.close()


######  5 - Limpeza e padronizacao dos dados

In [ ]:
# to do:
#valores de coluna (random/duplicar)
#evidencia
#trazer uma tabela final sem duplicadas

In [8]:
#renomeando os titulos das colunas
resultado1 = resultado1.rename(columns={'NOME': 'primeiro_nome', 
                                        'SOBRENOME': 'ultimo_nome', 
                                        'IDADE': 'nmr_idade', 
                                        'CIDADE': 'dsc_cidade', 
                                        'ESTADO': 'dsc_estado', 
                                        'PAIS': 'dsc_pais'})

In [9]:
#padronizando as informacoes
resultado1['primeiro_nome'] = resultado1['primeiro_nome'].str.lower()
resultado1['ultimo_nome'] = resultado1['ultimo_nome'].str.lower()
resultado1['dsc_cidade'] = resultado1['dsc_cidade'].str.lower()
resultado1['dsc_estado'] = resultado1['dsc_estado'].str.lower()
resultado1['dsc_pais'] = resultado1['dsc_pais'].str.lower()

In [10]:
#verificando os resultados
resultado1.tail()

,primeiro_nome,ultimo_nome,nmr_idade,dsc_cidade,dsc_estado,dsc_pais
12469,sindy,batista,25,altaneira,ce,brasil
12470,sindy,batista,25,altaneira,ce,brasil
12471,sindy,batista,25,altaneira,ce,brasil
12472,sindy,batista,25,altaneira,ce,brasil
12473,sindy,batista,25,altaneira,ce,brasil


######  6 - Categorizando faixa etaria e criacao da coluna flg_faixa_etaria 

In [11]:
#categorizando a faixa etaria
def categorise(row):  
    if row['nmr_idade'] >= 0 and row['nmr_idade'] <= 10:
        return '0 a 10'
    elif row['nmr_idade'] >= 11 and row['nmr_idade'] <= 20:
        return '11 a 20'
    elif row['nmr_idade'] >= 21  and row['nmr_idade'] <= 40:
        return '21 a 40'
    return '41+'

In [12]:
#criando uma nova coluna com as categorias da faixa etaria
resultado1['flg_faixa_etaria'] = resultado1.apply(lambda row: categorise(row), axis=1)

In [13]:
#verificando os resultados
resultado1.tail()

,primeiro_nome,ultimo_nome,nmr_idade,dsc_cidade,dsc_estado,dsc_pais,flg_faixa_etaria
12469,sindy,batista,25,altaneira,ce,brasil,21 a 40
12470,sindy,batista,25,altaneira,ce,brasil,21 a 40
12471,sindy,batista,25,altaneira,ce,brasil,21 a 40
12472,sindy,batista,25,altaneira,ce,brasil,21 a 40
12473,sindy,batista,25,altaneira,ce,brasil,21 a 40


######  7 - Ranking estado, cidade e flg faixa etaria

In [14]:
#ranking do estado, cidade e flg faixa etaria
top_estado = (resultado1[['dsc_estado']].value_counts().head())
top_cidade = (resultado1[['dsc_cidade']].value_counts().head())
top_flg_etaria = (resultado1[['flg_faixa_etaria']].value_counts().head())

In [15]:
#transformando em dataframe
top_estado = top_estado.to_frame()
top_cidade = top_cidade.to_frame()
top_flg_etaria = top_flg_etaria.to_frame()

In [16]:
top_estado

,0
dsc_estado,
es,1092
pr,960
ac,870
am,792
mt,700


In [17]:
top_cidade

,0
dsc_cidade,
apiacá,1092
anahy,960
rio branco,870
autazes,792
brasnorte,700


In [18]:
top_flg_etaria

,0
flg_faixa_etaria,
21 a 40,7748
11 a 20,2738
41+,1988


######  8 - Delete dos dados duplicados e deixando em ordem alfabetica

In [19]:
#deletando os dados duplicados e ordenando por ordem alfabetica
resultado1 = resultado1.drop_duplicates()
resultado1 = resultado1.sort_values('primeiro_nome', ascending=True)
resultado1 = resultado1.reset_index(drop=True)
resultado1

,primeiro_nome,ultimo_nome,nmr_idade,dsc_cidade,dsc_estado,dsc_pais,flg_faixa_etaria
0,carlos,garcia,36,aroazes,pi,brasil,21 a 40
1,dai,cardoso,38,apiacá,es,brasil,21 a 40
2,edson,barros,23,aiquara,ba,brasil,21 a 40
3,eduardo,dias,65,bela vista,ms,brasil,41+
4,edwin,costa,60,alfenas,mg,brasil,41+
5,erick,carvalho,44,aloândia,go,brasil,41+
6,fabio,castro,50,anapurus,ma,brasil,41+
7,francisco,freitas,18,aveiro,pa,brasil,11 a 20
8,helder,nunes,25,angelina,sc,brasil,21 a 40
9,jessica,alves,19,batalha,al,brasil,11 a 20


######  9 - Exportando o resultado em arquivo CSV

In [20]:
# exportando o resultado final em CSV
resultado1.to_csv ('resultado2.csv',sep=';',encoding='latin-1', index = False)